# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list

['/home/workspace/event_data/2018-11-30-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv',
 '/home/workspace/event_data/2018-11-14-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-28-events.csv',
 '/home/workspace/event_data/2018-11-25-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-12-events.csv',
 '/home/workspace/event_data/2018-11-27-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify 
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 2 };
"""
)


#### Set Keyspace

In [8]:
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
session.execute("""
CREATE TABLE IF NOT EXISTS music_by_session (
     sessionId int,
     itemInSession int,
     artist text,
     song text,
     length float,
     PRIMARY KEY(sessionId,itemInSession)
    )   
""")

In [32]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, _, _, itemInSession, _, length, _, _, sessionId, song, _ = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
rows = session.execute("""
    SELECT * from music_by_session
    LIMIT 10
    ;
"""
)

for record in rows:
    print(record)

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')
Row(sessionid=23, iteminsession=5, artist="Soulja Boy Tell'em", length=201.1162872314453, song='Let Me Get Em')
Row(sessionid=23, iteminsession=6, artist='Bodo Wartke', length=645.2763061523438, song='Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃ\x83Â¶sisch_ Italienisch_ Spanisch_ HollÃ\x83Â¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch)')
Row(sessionid=23, iteminsession=7, artist='Evanescence', l

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [44]:
session.execute("""
CREATE TABLE IF NOT EXISTS user_songs_by_session (
     userId int,
     sessionId int,
     itemInSession int,
     artist text,
     song text,
     first_name text, 
     last_name text,
     PRIMARY KEY ((userId, sessionId), itemInSession)
    )   
""")

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs_by_session (userId, sessionId, itemInSession, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist_name, first_name, _, itemInSession, last_name, _, _, _, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist_name, song, first_name, last_name))

In [46]:
rows = session.execute("""
    SELECT * from user_songs_by_session
    LIMIT 2
    ;
"""
)

for record in rows:
    print(record)

Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', first_name='Emily', last_name='Benson', song='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', first_name='Emily', last_name='Benson', song='Stranger Lover')


##  Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [47]:
session.execute("""
CREATE TABLE IF NOT EXISTS user_by_song (
     song text,
     userId int,
     first_name text, 
     last_name text,
     PRIMARY KEY(song, userId)
    )   
""")

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, userId, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        _, first_name, _, _, last_name, _, _, _, _, song, userId = line
        session.execute(query, (song, int(userId), first_name, last_name))

In [49]:
rows = session.execute("""
    SELECT * from user_by_song
    LIMIT 2
    ;
"""
)

for record in rows:
    print(record)

Row(song="Wonder What's Next", userid=49, first_name='Chloe', last_name='Cuevas')
Row(song="In The Dragon's Den", userid=49, first_name='Chloe', last_name='Cuevas')


### Drop the tables before closing out the sessions

In [51]:
for table in ['music_by_session', 'user_songs_by_session', 'user_by_song']:
    session.execute("""
        DROP TABLE IF EXISTS {}

        """.format(table)
        )

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()